# The One Goal for Today

Understand how to fit a naive Bayes model using text data.


In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt

# Naive Bayes for Spam

Text data, *without preprocessing*, is qualitative data. Let's use Naive Bayes to classify some text data! 

I'm going to be using the SMS dataset from [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## I. Load and Look at our data

Let's load and __look at our data__. Is our independent variable at the *start* or the *end*?

In [ ]:
data = np.array(np.genfromtxt('data/SMSSpamCollection', delimiter='\t', encoding='utf-8', dtype=str))  
print(data.shape)
print(data[2], "\n", data[4], "\n", data[9], "\n", data[10], "\n", data[11], "\n", data[12], "\n", data[14], "\n", data[20], "\n", data[29], "\n", data[32])

How many labels do we have?

In [ ]:
print(np.unique(data[:, 0], return_counts=True))

## II. Split the data

Let's split it into train, dev, test and make sure we have some of each label in each subset

In [ ]:
train, test = np.split(data, [int(.8 * len(data))])
print(train.shape, test.shape)
print(np.unique(train[:, 0], return_counts=True))
print(np.unique(test[:, 0], return_counts=True))

So far, so good!

## III. Preprocess the data

*But*, how do I deal with the SMS texts themselves? Well, a SMS is composed of word-type things (let's call them "tokens") and if I *completely ignore* the sequence information then I could say each SMS is composed of a *bag* of tokens. 

NOTE that to do this hurts me as a computational linguist. Of *course* word order matters! In particular, in this context it means that the token occurrences are not *independent* of each other. But we are going to pretend they are, thus lacerating the sensibilities of both linguists and statisticians in search of a computationally efficient approximation.

__Our independent variables will be what, now?__

### Let's define a function to get the bag of tokens from the text of a SMS

In [ ]:
# We will split the text on any character(s) that are not letters or numbers,
# and then keep anything that is not whitespace
def tokenize(text):
    # split on whitespace - \s+ - and strip off any whitespace
    return [token.strip() for token in re.split(r'(\s+)', text) if not re.match(r'^\s*$', token)]

In [ ]:
# Try it on one SMS
tokenize(train[0][1])

### Let's get the unique tokens

We do this on **train only**.

In [ ]:
# all the tokens
tokens = []
for i in range(len(train)):
    # add the tokens from this training data point to the list of all the tokens
    tokens.extend(tokenize(train[i][1]))
# get the unique tokens
unique_tokens, token_counts = np.unique(tokens, return_counts=True)
# how many do we have?
print(len(unique_tokens))
# what are some of them?
print(list(unique_tokens)[0:9])

That's entirely too many tokens! Let's keep just from the $a$ to the $b$ most frequent ones.

In [ ]:
a = 75
b = 2000
# all the tokens
tokens = []
for i in range(len(train)):
    # add the tokens from this training data point to the list of all the tokens
    tokens.extend(tokenize(train[i][1]))
# get the unique tokens
unique_tokens, token_counts = np.unique(tokens, return_counts=True)
# sort them by the counts; reverse the sort so it goes from largest to smallest; keep only the ones between a and b
top_unique_tokens = unique_tokens[np.argsort(token_counts)[::-1]][a:b]
print(len(top_unique_tokens))
print(top_unique_tokens[0:9])

### Let's collect token counts across all the 'ham' SMSs, and separately across all the 'spam' SMSs

In [ ]:
# subset_of_data means all the data points in a class / with the same value for the dependent variable
# token_vocab is the set of most frequent unique tokens we care about
def preprocess_category(subset_of_data, token_vocab):
    # the tokens in the data points in this class
    tokens = []
    for i in range(len(subset_of_data)):
        # add the tokens from this data point to tokens
        tokens.extend(tokenize(subset_of_data[i][1]))
    # get the unique tokens
    unique_tokens, token_counts = np.unique(tokens, return_counts=True)
    # we only want to keep those in token_vocab
    res = dict.fromkeys(token_vocab, 0)
    # add the count for each token that is in this class and that is a token we care about to res
    for token, count in zip(unique_tokens, token_counts):
        if token in res:
            res[token] += count
    return res

In [ ]:
token_counts = {}
# get token counts for ham
token_counts['ham'] = preprocess_category(train[np.where(train[:, 0] == 'ham')], top_unique_tokens)
# get token counts for spam
token_counts['spam'] = preprocess_category(train[np.where(train[:, 0] == 'spam')], top_unique_tokens)
# We keep this around for indexing into the likelihoods
sorted_tokens = sorted(top_unique_tokens)
print(len(sorted_tokens))

Pay attention to the sleight of hand here:
* Our labels are 'ham' and 'spam'.
* Our features are individual token frequencies calculated across all of 'ham' and all of 'spam'. We *assume independence in the features*. That's the naive in Naive Bayes.

## Fit

So now let's fit a Naive Bayes model for spam detection

To fit, we need to calculate:
* priors - $P(ham)$ and $P(spam)$
* likelihoods - for any token $x$, $P(x|ham)$ and $P(x|spam)$

### Priors

Calculating priors will be easy. 

In [ ]:
priors = {'ham': len(train[np.where(train[:, 0] == 'ham')]) / len(train), 'spam': len(train[np.where(train[:, 0] == 'spam')]) / len(train)}
print(priors)

### Likelihoods

Calculating likelihoods will be tricky, because:.
* Sometimes the likelihoods will be 0, as a token will occur *no* times in one of 'ham' or 'spam'.
* A lot of the likelihoods will be veerrrry small, as most of the tokens occur only once.

To deal with these, we:
* 0s - use *Laplace smoothing*.
* veerrrry small - move from regular count space to log space (see plot).
  * in 'regular count space' $P(I, like, candy) | ham) = P(I|ham)*P(like|ham)*P(candy|ham)$.
  * in 'log space' $P(I, like, candy) | ham) = log(P(I|ham)) + log(P(like|ham)) + log(P(candy|ham))$, because the logarithm of a product is the sum of the logarithms.

Let's visualize the difference between count space and log space.


In [ ]:
res = np.array([[i, 1/i, np.log(1/i)] for i in range(1, 1000)])
plt.plot(res[:, 0], res[:, 1])
plt.plot(res[:, 0], res[:, 2])
plt.show()

Let's calculate the log likelihoods using Laplace smoothing of 1.

In [ ]:
def calculate_likelihoods(data, sorted_tokens, token_counts):
    likelihoods = {}
    # assume we have all the possible values for the dependent variable in this data subset.
    cats = np.unique(data[:, 0])
    # for each possible outcome
    for cat in cats:
        likelihoods[cat] = {}
        # for each possible value of the independent variable
        for token in sorted_tokens:
            # add 1 in the numerator for Laplace smoothing; add the number of possible values of the independent variable to the denominator for Laplace smoothing
            # take the log of the likelihood
            likelihoods[cat][token] = np.log((token_counts[cat][token] + 1) / 
                                             (len(np.where([data[:, 0] == cat])) + len(sorted_tokens)))
    return likelihoods

In [ ]:
likelihoods = calculate_likelihoods(train, sorted_tokens, token_counts)
# sanity check!
print(len(likelihoods))
print(len(likelihoods['ham']))
print(len(likelihoods['spam']))

## Predict

Let's use the Naive Bayes model we fit to predict the class for each item in our test data.

We will have to tokenize each SMS, of course.

In [ ]:
def p_text_given_category(text, cat, likelihoods):
    tokens = tokenize(text)
    sum = 0
    for token in tokens:
        # if the token is not in our vocab from train, it just gets dropped
        try:
            likelihood = likelihoods[cat][token]
        except:
            likelihood = 0
        # we add instead of multiply, because we are in log space
        sum += likelihood
    return sum
        
def predict(data, priors, likelihoods):
    labels = []
    cats = np.unique(data[:, 0])
    for i in range(len(data)):    
        res = []
        for cat in cats:
            # we add instead of multiply, because we are in log space
            res.append(np.log(priors[cat]) + p_text_given_category(dev[i][1], cat, likelihoods))
        # take the argmax, and also its probability - take the exponent to get us out of log space
        labels.append((cats[np.argmax(res)], np.exp(res[np.argmax(res)])))
    return labels

In [ ]:
labels = predict(dev, priors, likelihoods)
print(labels)

## Score

Let's get accuracy and the confusion matrix.

In [ ]:
# These come directly from day 24

def accuracy(y, yhat):
    assert len(y) == len(yhat)
    diffs = y == yhat
    vals, counts = np.unique(diffs, return_counts=True)
    return counts[np.where(vals == True)] / (np.sum(counts))
    
def confusion_matrix(y, yhat):
    "Thanks to https://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python"
    allLabels = sorted(list(np.union1d(y, yhat)))
    print(allLabels)
    assert len(y) == len(yhat)
    result = np.zeros((len(allLabels), len(allLabels)))
    for i in range(len(y)):
        result[allLabels.index(y[i])][allLabels.index(yhat[i])] += 1
    return result

In [ ]:
accuracy([x[0] for x in labels], dev[:, 0])

In [ ]:
confusion_matrix([x[0] for x in labels], dev[:, 0])

Note that accuracy isn't super helpful here, since there are so many 'ham' emails. 

Question: *What would the accuracy be if the model just labeled __everything__ as 'ham'?*

## TPR, FPR, Precision, Recall, F

Let's make a different version of the confusion matrix focusing on just the 'spam' class, which is the class we really want to do well on:

| | Predict not in 'spam' | Predict in 'spam' | Rates |
| -- | --- | --- | -- |
| Actual not in 'spam' | TN | FP | FPR = FP/(FP+TN) |
| Actual in 'spam' | FN | TP | TPR = TP/(TP+FN) |

With a table like this, we can also calculate:
* Precision (how many of those we guessed were 'spam' were actually 'spam'?): TP / (TP + FP)
* Recall (how many actual 'spam' did we guess were 'spam'?): TP / (TP + FN)

Then, to get an assessment of recall and precision together, we can calculate F1: (2\*Precision\*Recall)/(Precision+Recall).

## Are we happy?

We could potentially do better:
* We could add information from *outside the text*. Modern spam filters use features like whether the sender's email address is the same as the reply-to email address, whether the IP address the email comes from matches that of the domain of the sender's email address, whether this same email is being sent to a lot of people at the same time, whether this sender has ever emailed this recipient before (and how often), and all kinds of behavioral features.
* We have a *class imbalance*: there are a lot more 'ham' emails than there are 'spam' emails. We can deal with this by *changing the decision threshold*, *downsampling* the 'ham' emails, *upsampling* (repeating) the 'spam' emails, or [other techniques](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/?msclkid=33d6b961b9df11ec98e519046a973ddf).

In [ ]:
my_texts = np.array([['ham', 'Thank you!'], ['spam', 'hey how are things'], ['ham', 'How is second puppy training going?'], ['ham', 'Your Clear Enrollment verification code is xxxx']])
pred = predict(my_texts, priors, likelihoods)
print(pred)

# For the project

For the project, you will need to load all the files in the ham folder, and all the files in the spam folder. You may use the python **glob** functionality for this. 
* For each directory in the enron directory (ham, spam):
  * For each .txt file in the directory:
    * Read the text in the file
    * Tokenize the text
    * For each token:
      * If it's not in the dictionary of token counts, add it there
      * Increment the count

For the project, you will keep only the 200 most frequent tokens.

Then, you will encode each headline uas a 200 dimensional vector (one column per token), and the labels as 0 or 1 for each headline (0 for spam, 1 for ham).

Then, you will want to split the data into train and test, and save the training and test data into separate files.

The whole process will look very similar to what we have just done.

Resources:
* https://towardsdatascience.com/roc-curve-and-auc-from-scratch-in-numpy-visualized-2612bb9459ab
* https://towardsdatascience.com/roc-and-auc-how-to-evaluate-machine-learning-models-in-no-time-fb2304c83a7f